In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def transform1(series):
    return series
def transform2(series):
    a = series.diff().fillna(0)
    a[1] = 0
    return a
def transform3(series):
    a = 2* series.diff().diff().fillna(0)
    return a
def transform4(series):
    return pd.Series(np.log(series))
def transform5(series):
    a = pd.Series(np.log(series)).diff().fillna(0)
    return a
def transform6(series):
    a = pd.Series(np.log(series)).diff().diff().fillna(0)
    return a
def transform7(series):
    a = series.div(series.add(-1).shift(-1)).fillna(0)
    return a

functions = [transform1, transform2, transform3, transform4, transform5, transform6, transform7]

def get_transformed_data(path):
    df = pd.read_csv(path)
    df2  = {}
    for name in df.columns:
        if(df[name].isna().sum() > 0 or name == 'sasdate'):
            continue
        df2[name] = functions[int(df[name][0])-1](df[name][1:])
        for i in range(1,4):
            df2[name+"_sb"+str(i)] = df2[name].shift(-1*i).fillna(0)

    df2 = pd.DataFrame(data=df2)
    target = df2['CPIULFSL']
    for name in df2.columns:
        if(np.abs(np.corrcoef(target, df2[name]))[0,1] > 0.2):
            print(name)
    
    

PPIFGS
PPIFCG
PPIITM
PPICRM
OILPRICEx
OILPRICEx_sb1
CPIAUCSL
CPITRNSL
CPITRNSL_sb2
CUSR0000SAC
CUSR0000SAS
CPIULFSL
CPIULFSL_sb1
CUSR0000SA0L2
CUSR0000SA0L5
PCEPI
DNDGRG3M086SBEA


In [73]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
y_train = df2['CPIULFSL'].shift(1).fillna(0)
x_train = df2.drop(columns=['CPIULFSL'])

l = Lasso()
l.fit(x_train, y_train)
l.score(x_train, y_train)

0.0